# Hands-on: Numerical Python -- Introduction to NumPy (Rehearsed and Continued)

**Objectives:**

This lesson is in large a rehearsal of the introduction video we watched last week, but now with practical excercises and extending on ideas of accessing and manipulating data with NumPy arrays.  Upon completion of this class, you will be able to

1. Access data in N-dimensional arrays via indexing, slicing, fancy indexing

2. Perform various operations on the arrays, and become aware that some times you get a "view", some times a "copy"

3. Practice broadcasting

## Indexing and slicing



The items of an array can be accessed and assigned to the same way as
other Python sequences (`list`, `tuple`):

In [ ]:
import numpy as np

a = np.arange(10)
print(a)
print(a[0], a[2], a[-1])

**Note**: Indices begin at 0, like other Python sequences (and C/C++): In contrast, in Fortran or Matlab, indices begin at 1.

## Multidimensional arrays



Indexes are tuples of integers:

In [ ]:
a = np.diag(np.arange(5))
a

In [ ]:
a[1, 1]

In [ ]:
a[2, 1] = 10 # third line, second column
a

and specifying only one index, would select the corresponding 'row' or 'column':

In [ ]:
a[1]

In [ ]:
a[:, 1]

in above example I have used ":" to specify "all" elements in that axis.  Alternatively,  even if we had >2 axes, I could use "..." to specify "all previous axes having ":"

In [ ]:
a[..., 1]

## Slicing



Arrays, like other Python sequences can also be sliced:

In [ ]:
a = np.arange(10)
a

In [ ]:
a[2:9:3] # [start:end:step]

## Indexing and Slicing

A small illustrated summary of Numpy indexing and slicing...

![NumPy slicing and indexing](http://scipy-lectures.github.io/_images/numpy_indexing.png)

### Fancy indexing

*Fancy indexing* is just a fancy name for indexing multiple entries at once by either providing indexes of the entries:

In [ ]:
a = np.array(list('abcdefgh'))
print(a[[0, 2, 4, 6]])

or a boolean mask:

In [ ]:
a = np.array(list('abcdefgh'))
print(a[[True, False]*4])

**There was/used to be a catch**:  bool mask had to be an array of type **bool**!  **Excercise**: find when behavior changed

### Assigning sliced/fancy indexed array elements:

Very powerful feature to modify in-place some elements identified by indexes or a mask:

In [ ]:
a[[0, 2]] = 'X'
print(a)

In [ ]:
a[a == 'g'] = 'Y'
print(a)

## Array's "internal" flags

`.flags` of any array provide information about the "internals" of the array.  Lets look at them:

In [ ]:
a_slice = a[::2]
print(a_slice)
print(a_slice.flags)

In [ ]:
a_findexed = a[[0, 2, 4, 6]]
print(a_findexed)
print(a_findexed.flags)

See [setflags documentation](http://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.setflags.html#numpy.ndarray.setflags) for more details about flags, but for now we concentrate on "OWNDATA".

- - -
**Excercise**

`np.random` provides functions to generate random arrays.  Create a 2d array of shape `(3, 4)` with normally distributed data.  Experiment with slicing, indexing and fancy indexing.  Check `OWNDATA` flag of created arrays.

## Elementwise operations



With scalars:

In [ ]:
a = np.array([1, 2, 3, 4])
a + 1

In [ ]:
2**a

All arithmetic operates elementwise (like ufuncs)

In [ ]:
b = np.ones(4) + 1
a - b

In [ ]:
a * b

In [ ]:
j = np.arange(5)
2**(j + 1) - j

**Warning!** Multiplication is no special and is also elementwise:

In [ ]:
c = np.ones((3, 3))
c * c                   # NOT matrix multiplication!

Matrix multiplication:

In [ ]:
c.dot(c)

**Note** Since python3. we get a new operation symbol "@" specifically reserved for "matrix multiplication".

In [ ]:
c @ c

### Speeding up some computations

There is a growing number of libraries which try to take even more advantage of existing hardware features (e.g. GPUs, multiple CPUs etc) or simply providing even more efficient implementations (reuse of memory, etc).  E.g. [numexpr](https://code.google.com/p/numexpr/wiki/UsersGuide) could speed up majority of ufunc executions and even `np.where`:

In [ ]:
a = np.random.normal(1e6)
b = np.arange(1e6)
c = a**2 + 2*b**3 + 2*a*b
pos_c = c>0

In [ ]:
# The equation
%timeit a**2 + 2*b**3 + 2*a*b
%timeit np.where(pos_c, 1, 2)

In [ ]:
import numexpr as ne
%timeit ne.evaluate("a**2 + 2*b**3 + 2*a*b")
%timeit ne.evaluate("where(pos_c, 1, 2)")

In [ ]:
# now use a single thread
ne.set_num_threads(1)
%timeit ne.evaluate("a**2 + 2*b**3 + 2*a*b")
%timeit ne.evaluate("where(pos_c, 1, 2)")

## Comparisons



You can make fast comparisons of ndarrays:

In [ ]:
a = np.array([1, 2, 3, 4])
b = np.array([4, 2, 2, 4])
a == b

In [ ]:
a > b

so note that it is a unary functionality, i.e. it compares each element at a time, and unlike (**which stock container?**) doesn't result in a single **bool** value:

In [ ]:
if a > b:
    print("a is bigger")

and you need to use **numpy.any** or **numpy.all** (well -- regular **all** and **any** would work but slower) `reductions` to get the target **bool**:

In [ ]:
print(np.all(a > b))

In [ ]:
print(np.any(a > b))

you could also use `array_equal` helper

In [ ]:
np.array_equal(a, b)

- - -
**Questions**

- There is also `np.allclose` -- check out its help.  When will it be useful?

## Logical operations



And perform fast logical operations:

In [ ]:
a = np.array([1, 1, 0, 0], dtype=bool)
b = np.array([1, 0, 1, 0], dtype=bool)
a | b

In [ ]:
a & b

**Note**: For arrays: "`&`" and "`|`" for logical operations, not: "`and`" and "`or`".  You could also use **numpy.logical_and** and **numpy.logical_or** functions.

## Shape mismatches



What if things don't line up?

In [ ]:
a = np.array([0.1, 0.6, -0.3])
b = np.array([1, 2])
print(a + b)

### Broadcasting

Broadcasting could be of great use.  It follows 3 simple steps:

1. If array shapes differ, left-pad the *smallest* shape with 1s
2. If any dimension does not match, broadcast the dimension with size=1
   - **Question**: Does it have to be a single non-matching dimension or could be many?
3. If neither non-matching dimension is 1 -- raise an Error
   - **Question**: Which error gets raised?


[Scientific Python Lecture Notes](http://scipy-lectures.github.io/intro/numpy/operations.html#broadcasting):
![scipy Broadcasting](http://scipy-lectures.github.io/_images/numpy_broadcasting.png)

In [ ]:
a = np.tile(np.arange(0, 40, 10), (3, 1)).T
print(a)
b = np.array([0, 1, 2])
print(b)

In [ ]:
print(a + b)

We could left-pad with new dimension manually:

In [ ]:
print(b.shape)
print(b[np.newaxis, :].shape)
print(a + b[np.newaxis, :])

- - -

**Broadcasting excercise**

Assuming that in above failed example I wanted to generate 3x2 array in which first column will be a+1 and 2nd column would be a+2 -- how to adjust above addition to make it happen? (hint: we need to introduce one more axis to one of those 1d vectors)

## More of explicit array shape manipulations

### Simple transpose

In [ ]:
a_T = a.T
print(a_T)

Such a transpose is very fast operation since no data copying is actually being done -- only a new "view" created over the array data:

In [ ]:
print(a_T.flags)

so similarly to "sliced" array, changing element in the copy would change data in the original array:

In [ ]:
a_T[1, 0] = 101
print(a_T)
print(a)

### Flattening

Create a flattened version of an array, with the highest dimension `ravel`ing first:

In [ ]:
a_flat = a.ravel()
print(a_flat)

- - -
**Excercises**
1. Read help of `np.ravel` about two possible different "orders".
2. Access `a.flags` to see which order `a` is in
3. Flatten `a` in its "original" order and store result into `a_flatten`
4. Acess `a_flatten.flags`.  What does OWNDATA would mean?  Try changing some element within `a_flatten` and inspect content of original `a`


If we do not care to get a "view" over an array, we can use `.flatten()` method which is guaranteed to return a copy:

In [ ]:
a.flatten()

### Reshaping

In [ ]:
a_3d = a.reshape((2, 2, 3))
print(a_3d.shape)
print(a_3d)

- - -
**Excercise**
1. Inspect `a_3d` -- have we created a copy or a view?
2. Checkout `np.reshape` help

**Question**

If in the case of `.T` operation or slicing we can somewhat rely that we would obtain a 'view', it is not the fact with `ravel` and `reshape`.  How should we "guard" our code to state our expectation that we got a copy or not?

## Arrays concatenations

There is a number of functions to "join" multiple arrays or repeat existing ones multiple times:

In [ ]:
a = np.arange(4).reshape((2, 2))
b = np.arange(4,6)
print(a, a.shape)
print(b, b.shape)

In [ ]:
c = np.vstack((a, b))
print(c, c.shape)

In [ ]:
c = np.hstack((a, b[:, np.newaxis]))
print(c, c.shape)

or an array could be repeated (by default flattened)

In [ ]:
print(np.repeat(a, 2))

or along a given axis

In [ ]:
np.repeat(a, 2, axis=1)

possibly even with varying number of repeats per each element (along that axis):

In [ ]:
np.repeat(a, (2, 3), axis=1)

- - -
**Excercise**

Imagine we are to carry out an fMRI experiment where we have 

- 3 conditions: 2 experimental conditions (let's code them as 1 and 2) and a baseline (code as 0)
- we will have 4 instances (trials) of each condition in each run
- order of trials must be random (run can start with any) (hint: use `np.random.shuffle`)
- trials will span varying number of TRs for each condition:  0 - 1 TR,  1 - 2TRs,  2 - 3TRs
- **generate a sequence of TRs for such an experimental run**
- verify that we have create "proper" sequence in terms of how often each condition was present (hint: discover `np.bincount` function)

## Basic linear algebra



Matrix multiplication:

In [ ]:
a = np.triu(np.ones((3, 3)), 1)   # see help(np.triu)
print(a)

In [ ]:
b = np.diag([1, 2, 3])
print(a.dot(b))

In [ ]:
print(np.dot(a, a))

###   Inverses and linear equation systems:

In [ ]:
A = a + b
print(A)
B = np.linalg.inv(A)
print(B)

In [ ]:
B.dot(A)

###   Eigenvalues:

In [ ]:
np.linalg.eigvals(A)

... and so on, see

In [ ]:
?np.linalg

## Basic reductions



Computing sums:

In [ ]:
x = np.array([1, 2, 3, 4])
print(np.sum(x))
print(x.sum())

### Sum by rows and by columns

![Sum across rows and columns](http://scipy-lectures.github.io/_images/reductions.png)

In [ ]:
x = np.array([[1, 1], [2, 2]])
print(x)

We can ask for an operation along specific axis:

In [ ]:
x.sum(axis=0)   # columns (first dimension)

which is identical to having done something like:

In [ ]:
x[:, 0].sum(), x[:, 1].sum()

and we could easily sum within each row:

In [ ]:
x.sum(axis=1)   # rows (second dimension)

In [ ]:
x[0, :].sum(), x[1, :].sum()

## Other reductions



Work the same way (and take `axis=`)

* Statistics:

In [ ]:
x = np.array([1, 2, 3, 1])
y = np.array([[1, 2, 3], [5, 6, 1]])
x.mean()

In [ ]:
np.median(x)

In [ ]:
np.median(y, axis=-1) # last axis

In [ ]:
x.std()          # full population standard dev.

In [ ]:
x.std(ddof=1)    # sample std (with N-1 in divisor)



* 
Extrema:

In [ ]:
x = np.array([1, 3, 2])
x.min(), x.max()

In [ ]:
# indexes of (min, max)
x.argmin(), x.argmax()

* ... and many more (best to learn as you go).

## Sorting arrays

`np.sort` provides a new array with entries sorted within any given axis:

In [ ]:
a = np.array([[4, 3, 5], [1, 2, 1]])
b = np.sort(a, axis=1)
print(a)
print(b)

But we could also sort "inplace", i.e. by modifying the original array:

In [ ]:
a.sort(axis=1)
print(a)

- - -

**Excercise**

1. Create an array of shape (3, 4) with entries from 0 to 11 randomly assigned to different elements.  For that make use of some functions you already know, and `np.random.shuffle`
2. Sort each row
3. Assuming that we are having an ultimate trust into built-in `sorted` function, write a code snippet which would verify that you have sorted your rows correctly (and raises AssertionError if not)

## NumPy testing helper

NumPy comes with a `.testing` submodule which provides handy utilities for unit-testing your code where you need to verify correct operation on NumPy arrays, e.g.

In [ ]:
import numpy.testing as npt

In [ ]:
npt.assert_array_less(np.array([1,2,3]), np.array([2,3,4]))

In [ ]:
npt.assert_array_equal(np.array([1,2,3]), np.array([1., 2., 3.]))

In [ ]:
npt.assert_array_almost_equal(np.array([1,2,3]), np.array([1.0000001, 2., 3.]))

- - -
**Excercise**

Adjust your code snippet in previous excercise to use numpy.testing assertion helpers

## More on NumPy data types

So far we already saw that we could store integers, floating point numbers, and even strings into numpy arrays.  `.dtype` provides us description of the data type of the array:

In [ ]:
np.arange(3).dtype

In [ ]:
np.ones(3).dtype

In [ ]:
np.array(['a', 'bc']).dtype

so numpy usually figures out the appropriate data-type but at times we might want to specify it explicitly, e.g. to save some memory (and possibly CPU time) at the cost of precision

In [ ]:
np.ones(3, dtype=np.float32).dtype

and you can "cast" your existing array into another compatible type:

In [ ]:
np.ones(3, dtype=np.float32).astype(int)

If you need to discover details (max/min value, resolution) of the specific floating point number data type, use `finfo` function

In [ ]:
np.finfo(np.float32)

**Extra**

For hungry minds -- checkout [Structured data types](http://scipy-lectures.github.io/intro/numpy/elaborate_arrays.html#structured-data-types) and [Masked arrays](http://scipy-lectures.github.io/intro/numpy/elaborate_arrays.html#id3)